## BiLSTM for MIT Movies
I was going to make this repository a package with setup.py and everything but because of my deadlines and responsibilities at my current workplace I haven't got the time to do that so I shared the structure of the project in README.md file.

In [1]:
import os
import sys
import json
import numpy as np
from tqdm import tqdm
sys.path.append("..")

from torch import nn
from torch.optim import Adam
from src.movientities.models.lstm import BiLSTM
from src.movientities.utils.processors import NerPreProcessor
from src.movientities.trainers import TrainerBiLstm
from src.movientities.data.build_dataset import Corpus, BuildData

In [2]:
# Create datasets with tab as splitter for corpus of torch text to handle
BuildData.create_finaldata("/home/kemalaraz/Desktop/MovieEntityRecognizer/data/raw/mitmovies/engtest.bio", "/home/kemalaraz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format/test.txt", splits="\t")
BuildData.create_finaldata("/home/kemalaraz/Desktop/MovieEntityRecognizer/data/raw/mitmovies/engtest.bio", "/home/kemalaraz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format/test.txt", splits="\t")

In [3]:
dataset = Corpus(
    input_folder="/home/kemalaraz/Desktop/MovieEntityRecognizer/data/modified/mitmovies_tab_format",
    min_word_freq=3,  # any words occurring less than 3 times will be ignored from vocab
    batch_size=64)
print(f"Train set: {len(dataset.train_dataset)} sentences")
print(f"Test set: {len(dataset.test_dataset)} sentences")

Train set: 9775 sentences
Test set: 2443 sentences


In [10]:
model = BiLSTM(
    input_dim=len(dataset.word_field.vocab),
    embedding_dim=300,
    hidden_dim=64,
    output_dim=len(dataset.tag_field.vocab),
    lstm_layers=4,
    emb_dropout=0.25,
    lstm_dropout=0.01,
    fc_dropout=0.1,
    word_pad_idx=dataset.word_pad_idx,
)
# Initialize weights and embeddings
model.init_weights()
model.init_embeddings(word_pad_idx=dataset.word_pad_idx)
print(f"The model has {model.count_parameters():,} trainable parameters.")
print(model)

The model has 1,161,930 trainable parameters.
BiLSTM(
  (embedding): Embedding(2244, 300, padding_idx=1)
  (emb_dropout): Dropout(p=0.25, inplace=False)
  (lstm): LSTM(300, 64, num_layers=4, dropout=0.01, bidirectional=True)
  (fc_dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=128, out_features=26, bias=True)
)


In [12]:
ner = TrainerBiLstm(
  model=model,
  data=dataset,
  optimizer_cls=Adam,
  loss_fn_cls=nn.CrossEntropyLoss,
  log_name="bilstm_vanilla2"
)
ner.train(20)

Epoch: 01 | Epoch Time: 0m 22s
	Trn Loss: 0.903 | Trn Acc: 75.58%
	Val Loss: 0.763 | Val Acc: 80.36% | Val Precision: 60.83% | Val Recall: 55.18% | Val F1 Macro: 53.84% | Val F1 Micro: 80.28%
Epoch: 02 | Epoch Time: 0m 22s
	Trn Loss: 0.448 | Trn Acc: 88.92%
	Val Loss: 0.520 | Val Acc: 87.07% | Val Precision: 74.27% | Val Recall: 72.57% | Val F1 Macro: 70.96% | Val F1 Micro: 86.40%
Epoch: 03 | Epoch Time: 0m 22s
	Trn Loss: 0.321 | Trn Acc: 92.06%
	Val Loss: 0.453 | Val Acc: 88.60% | Val Precision: 75.65% | Val Recall: 75.65% | Val F1 Macro: 73.28% | Val F1 Micro: 87.79%
Epoch: 04 | Epoch Time: 0m 21s
	Trn Loss: 0.272 | Trn Acc: 93.18%
	Val Loss: 0.432 | Val Acc: 88.86% | Val Precision: 77.34% | Val Recall: 73.99% | Val F1 Macro: 73.20% | Val F1 Micro: 87.94%
Epoch: 05 | Epoch Time: 0m 22s
	Trn Loss: 0.238 | Trn Acc: 94.04%
	Val Loss: 0.415 | Val Acc: 89.34% | Val Precision: 74.38% | Val Recall: 74.84% | Val F1 Macro: 72.41% | Val F1 Micro: 88.23%
Epoch: 06 | Epoch Time: 0m 21s
	Trn Loss

20 Epoch Adam lr 0.001lstm drop 0.1 fc dopr 0.25 batch 64

Epoch: 01 | Epoch Time: 0m 29s
	Trn Loss: 1.658 | Trn Acc: 61.69%
	Val Loss: 1.227 | Val Acc: 66.63% | Val Precision: 31.96% | Val Recall: 52.00% | Val F1 Macro: 36.41% | Val F1 Micro: 72.99%
Epoch: 02 | Epoch Time: 0m 34s
	Trn Loss: 0.845 | Trn Acc: 76.78%
	Val Loss: 0.753 | Val Acc: 80.34% | Val Precision: 57.47% | Val Recall: 55.16% | Val F1 Macro: 52.33% | Val F1 Micro: 80.03%
Epoch: 03 | Epoch Time: 0m 33s
	Trn Loss: 0.538 | Trn Acc: 86.26%
	Val Loss: 0.600 | Val Acc: 85.45% | Val Precision: 70.97% | Val Recall: 69.60% | Val F1 Macro: 67.31% | Val F1 Micro: 84.88%
Epoch: 04 | Epoch Time: 0m 33s
	Trn Loss: 0.412 | Trn Acc: 89.78%
	Val Loss: 0.476 | Val Acc: 88.51% | Val Precision: 73.39% | Val Recall: 75.89% | Val F1 Macro: 72.66% | Val F1 Micro: 87.82%
Epoch: 05 | Epoch Time: 0m 34s
	Trn Loss: 0.346 | Trn Acc: 91.46%
	Val Loss: 0.458 | Val Acc: 88.67% | Val Precision: 73.57% | Val Recall: 73.83% | Val F1 Macro: 71.65% | Val F1 Micro: 87.77%
Epoch: 06 | Epoch Time: 0m 33s
	Trn Loss: 0.305 | Trn Acc: 92.39%
	Val Loss: 0.432 | Val Acc: 89.27% | Val Precision: 76.20% | Val Recall: 74.37% | Val F1 Macro: 73.06% | Val F1 Micro: 88.32%
Epoch: 07 | Epoch Time: 0m 25s
	Trn Loss: 0.275 | Trn Acc: 93.19%
	Val Loss: 0.435 | Val Acc: 89.10% | Val Precision: 77.82% | Val Recall: 75.45% | Val F1 Macro: 74.63% | Val F1 Micro: 88.10%
Epoch: 08 | Epoch Time: 0m 29s
	Trn Loss: 0.256 | Trn Acc: 93.59%
	Val Loss: 0.432 | Val Acc: 89.23% | Val Precision: 77.96% | Val Recall: 75.78% | Val F1 Macro: 74.90% | Val F1 Micro: 88.20%
Epoch: 09 | Epoch Time: 0m 31s
	Trn Loss: 0.233 | Trn Acc: 94.13%
	Val Loss: 0.407 | Val Acc: 90.06% | Val Precision: 78.80% | Val Recall: 77.47% | Val F1 Macro: 76.32% | Val F1 Micro: 88.94%
Epoch: 10 | Epoch Time: 0m 27s
	Trn Loss: 0.217 | Trn Acc: 94.55%
	Val Loss: 0.394 | Val Acc: 90.39% | Val Precision: 80.00% | Val Recall: 79.13% | Val F1 Macro: 77.82% | Val F1 Micro: 89.33%
Epoch: 11 | Epoch Time: 0m 25s
	Trn Loss: 0.205 | Trn Acc: 94.79%
	Val Loss: 0.396 | Val Acc: 90.41% | Val Precision: 78.76% | Val Recall: 77.90% | Val F1 Macro: 76.47% | Val F1 Micro: 89.24%
Epoch: 12 | Epoch Time: 0m 24s
	Trn Loss: 0.191 | Trn Acc: 95.05%
	Val Loss: 0.428 | Val Acc: 89.46% | Val Precision: 77.10% | Val Recall: 76.90% | Val F1 Macro: 74.92% | Val F1 Micro: 88.31%
Epoch: 13 | Epoch Time: 0m 25s
	Trn Loss: 0.179 | Trn Acc: 95.34%
	Val Loss: 0.392 | Val Acc: 90.74% | Val Precision: 80.24% | Val Recall: 79.21% | Val F1 Macro: 78.04% | Val F1 Micro: 89.60%
Epoch: 14 | Epoch Time: 0m 26s
	Trn Loss: 0.170 | Trn Acc: 95.51%
	Val Loss: 0.419 | Val Acc: 90.37% | Val Precision: 79.34% | Val Recall: 79.72% | Val F1 Macro: 77.80% | Val F1 Micro: 89.25%
Epoch: 15 | Epoch Time: 0m 24s
	Trn Loss: 0.164 | Trn Acc: 95.68%
	Val Loss: 0.392 | Val Acc: 90.65% | Val Precision: 79.81% | Val Recall: 79.83% | Val F1 Macro: 78.28% | Val F1 Micro: 89.53%
Epoch: 16 | Epoch Time: 0m 26s
	Trn Loss: 0.154 | Trn Acc: 95.89%
	Val Loss: 0.404 | Val Acc: 90.56% | Val Precision: 79.85% | Val Recall: 78.63% | Val F1 Macro: 77.54% | Val F1 Micro: 89.40%
Epoch: 17 | Epoch Time: 0m 26s
	Trn Loss: 0.148 | Trn Acc: 95.99%
	Val Loss: 0.400 | Val Acc: 90.59% | Val Precision: 78.39% | Val Recall: 77.84% | Val F1 Macro: 76.49% | Val F1 Micro: 89.40%
Epoch: 18 | Epoch Time: 0m 28s
	Trn Loss: 0.140 | Trn Acc: 96.25%
	Val Loss: 0.434 | Val Acc: 90.37% | Val Precision: 79.29% | Val Recall: 78.36% | Val F1 Macro: 77.08% | Val F1 Micro: 89.21%
Epoch: 19 | Epoch Time: 0m 29s
	Trn Loss: 0.135 | Trn Acc: 96.33%
	Val Loss: 0.402 | Val Acc: 91.19% | Val Precision: 80.33% | Val Recall: 78.81% | Val F1 Macro: 77.74% | Val F1 Micro: 89.98%
Epoch: 20 | Epoch Time: 0m 24s
	Trn Loss: 0.126 | Trn Acc: 96.57%
	Val Loss: 0.381 | Val Acc: 91.04% | Val Precision: 78.34% | Val Recall: 77.63% | Val F1 Macro: 76.52% | Val F1 Micro: 89.79%

